# Recycling in Europe - Data Preprocessing Pipeline

This notebook processes and cleans multiple datasets related to recycling rates and environmental factors across European countries. The datasets are combined to create a unified dataset for analysis.

## Overview of Datasets:
- **D1**: General recycling rates (overall waste recycling %)
- **D2**: Recycling rates by waste type (glass, plastic, paper, metallic, wooden, packaging)
- **D3**: Socioeconomic indicators (GDP per capita, urbanization, internet usage, renewable energy, etc.)
- **CEI**: Circular economy indicators (private investments and output of circular economy sectors)
- **ENV**: Environmental tax revenues (as % of GDP)


# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
MISSING_RATE = 0.9
EU_COUNTRIES = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 
    'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 
    'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 
    'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 
    'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 
    'Spain', 'Sweden'
]

# Preprocessing

## Functions

In [ ]:
def fill_missing(series: pd.Series) -> pd.Series:
    # 1. interpolation for internal gaps
    # 2. backward fill for leading NaNs
    # 3. forward fill for trailing NaNs
    return series.interpolate().bfill().ffill()

## D1 - general recylcing rates

In [ ]:
d1 = pd.read_csv("raw_data/d1.csv")

In [ ]:
res_d1 = d1.copy()
res_d1 = res_d1[['geo', 'TIME_PERIOD', "OBS_VALUE", "OBS_FLAG"]]
res_d1 = res_d1.rename(columns={"geo": "country_name",
                                "TIME_PERIOD": "year",
                                "OBS_VALUE": "recycling_rate",
                                "OBS_FLAG": "flag"})
res_d1['country_name'] = res_d1['country_name'].astype("string")
res_d1['year'] = res_d1['year'].astype(int)
res_d1['recycling_rate'] = res_d1['recycling_rate'].astype(float)
res_d1['flag'] = res_d1['flag'].astype("string")

missing_rates_d1 = pd.DataFrame(res_d1.groupby("country_name")["recycling_rate"].apply(lambda x: x.isna().mean())).reset_index()
missing_rates_d1.to_csv("processed_data/missing_value_rates_d1.csv", index=False)

values_to_drop_d1 = []
for index, row in missing_rates_d1.iterrows():
    if row['recycling_rate'] > MISSING_RATE:
        values_to_drop_d1.append(row['country_name'])
res_d1 = res_d1[~res_d1['country_name'].isin(values_to_drop_d1)]
print(f"Dropped countries in d1 (n={len(values_to_drop_d1)}):", values_to_drop_d1)

res_d1['recycling_rate_filled'] = res_d1.groupby("country_name")["recycling_rate"].transform(fill_missing)

In [ ]:
res_d1.isna().sum()

In [ ]:
res_d1.to_csv("processed_data/preprocessed_d1.csv", index=False)

## D2 - different types of waste

In [ ]:
d2 = pd.read_csv("raw_data/d2.csv")

In [ ]:
res_d2 = d2.copy()
res_d2 = res_d2[['geo', 'TIME_PERIOD', 'waste', 'OBS_VALUE', 'OBS_FLAG']]
res_d2 = res_d2.rename(columns={"geo": "country_name",
                                "TIME_PERIOD": "year",
                                "waste": "waste_type",
                                "OBS_VALUE": "recycling_rate",
                                "OBS_FLAG": "flag"})
res_d2['country_name'] = res_d2['country_name'].astype("string")
res_d2['year'] = res_d2['year'].astype(int)
res_d2['waste_type'] = res_d2['waste_type'].astype("string")
res_d2['recycling_rate'] = res_d2['recycling_rate'].astype(float)
res_d2['flag'] = res_d2['flag'].astype("string")

missing_rates_d2 = pd.DataFrame(res_d2.groupby(["country_name", 'waste_type'])["recycling_rate"].apply(lambda x: x.isna().mean())).reset_index()
missing_rates_d2.to_csv("processed_data/missing_value_rates_d2.csv", index=False)

values_to_drop_d2 = set()
for index, row in missing_rates_d2.iterrows():
    country_name = row['country_name']
    waste_type = row['waste_type']
    rate = row['recycling_rate']
    if rate > MISSING_RATE:
        values_to_drop_d2.add(country_name)
        print(f"{country_name}\t{waste_type}")

res_d2 = res_d2[~res_d2['country_name'].isin(values_to_drop_d2)]
print(f"Dropped countries in d2 (n={len(values_to_drop_d2)}):", values_to_drop_d2)

res_d2['recycling_rate_filled'] = res_d2.groupby(["country_name", "waste_type"])["recycling_rate"].transform(fill_missing)

clean_names = {
    "Glass packaging": "glass",
    "Metallic packaging": "metallic",
    "Packaging": "packaging",
    "Paper and cardboard packaging": "paper",
    "Plastic packaging": "plastic",
    "Wooden packaging": "wooden"
}
res_d2['waste_type'] = res_d2['waste_type'].map(clean_names)

res_d2 = res_d2.pivot(index=['country_name', 'year'], columns='waste_type', values=['recycling_rate_filled', "recycling_rate", "flag"]).reset_index()
res_d2.columns = [f"{val}_{waste}" if waste else val for val, waste in res_d2.columns]

In [ ]:
res_d2.isna().sum()

In [ ]:
res_d2.to_csv("processed_data/preprocessed_d2.csv", index=False)

In [ ]:
res_d2['year'].min(), res_d2['year'].max()

## Checking country overlap between D1 and D2

In [ ]:
for c in res_d1['country_name'].unique():
    if c not in res_d2['country_name'].unique():
        print(c)

In [ ]:
for c in res_d2['country_name'].unique():
    if c not in res_d1['country_name'].unique():
        print(c)

In [ ]:
values_to_drop_d1

In [ ]:
values_to_drop_d2

In [ ]:
countries = set(res_d1['country_name'].unique()).union(set(res_d2['country_name'].unique()))

In [ ]:
countries

## D3

In [ ]:
d3 = pd.read_csv("raw_data/d3.csv")

In [ ]:
for c in countries:
    if c not in d3['Country Name'].unique():
        print(c)

In [ ]:
d3 = d3.replace("Slovak Republic", "Slovakia")
d3 = d3.replace("Kosovo", "Kosovo*")
d3 = d3.replace("Turkiye", "Türkiye")
for c in countries:
    if c not in d3['Country Name'].unique():
        print(c)

In [ ]:
d3 = d3[d3['Country Name'].isin(countries)]

In [ ]:
d3

In [ ]:
res_d3 = d3.copy()
res_d3 = res_d3.iloc[:-5, :]
res_d3 = res_d3.rename(columns={"Time": "year",
                                "Country Name": "country_name",
                                "GDP per capita (constant 2015 US$) [NY.GDP.PCAP.KD]": "gdp_per_capita",
                                "Urban population (% of total population) [SP.URB.TOTL.IN.ZS]": "urban_population_pct",
                                "Individuals using the Internet (% of population) [IT.NET.USER.ZS]": "internet_users_pct",
                                "Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS]": "renewable_energy_pct",
                                "International tourism, number of arrivals [ST.INT.ARVL]": "tourism_arrivals",
                                "Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]": "government_debt_pct_gdp",
                                "Population, total [SP.POP.TOTL]": "population_total",
                                "Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]": "manufacturing_value_added_pct_gdp",
                                "Government Effectiveness: Estimate [GE.EST]": "government_effectiveness_estimate"})
res_d3 = res_d3[["country_name", "year", "gdp_per_capita", "urban_population_pct",
                 "internet_users_pct", "renewable_energy_pct", "tourism_arrivals",
                 "government_debt_pct_gdp", "population_total", "manufacturing_value_added_pct_gdp",
                 "government_effectiveness_estimate"]]
res_d3 = res_d3.replace("..", np.nan)
res_d3['country_name'] = res_d3['country_name'].astype("string")
res_d3['year'] = res_d3['year'].astype(int)
res_d3['gdp_per_capita'] = res_d3['gdp_per_capita'].astype(float)
res_d3['urban_population_pct'] = res_d3['urban_population_pct'].astype(float)
res_d3['internet_users_pct'] = res_d3['internet_users_pct'].astype(float)
res_d3['renewable_energy_pct'] = res_d3['renewable_energy_pct'].astype(float)
res_d3['tourism_arrivals'] = res_d3['tourism_arrivals'].astype(float)
res_d3['government_debt_pct_gdp'] = res_d3['government_debt_pct_gdp'].astype(float)
res_d3['population_total'] = res_d3['population_total'].astype(float)
res_d3['manufacturing_value_added_pct_gdp'] = res_d3['manufacturing_value_added_pct_gdp'].astype(float)
res_d3['government_effectiveness_estimate'] = res_d3['government_effectiveness_estimate'].astype(float)

# # more than 20 european countries have all missing values in "government_debt_pct_gdp"
res_d3 = res_d3.drop(columns=["government_debt_pct_gdp"])


values_to_drop_d3 = []
missing_rates_d3 = pd.DataFrame()
missing_rates_d3['country_name'] = res_d3['country_name'].unique()
for col in res_d3.columns:
    if col in ["country_name", "year"]:
        continue
    missing_rates_d3_col = pd.DataFrame(res_d3.groupby("country_name")[col].apply(lambda x: x.isna().mean())).reset_index()
    missing_rates_d3 = pd.merge(missing_rates_d3, missing_rates_d3_col, on='country_name', how='left', suffixes=('', f'_{col}'))
    for index, row in missing_rates_d3_col.iterrows():
        if row[col] > MISSING_RATE:
            values_to_drop_d3.append(row['country_name'])
            print(f"{row['country_name']}\t{col}")
missing_rates_d3.to_csv("processed_data/missing_value_rates_d3.csv", index=False)
res_d3 = res_d3[~res_d3['country_name'].isin(values_to_drop_d3)]


for col in res_d3.columns:
    if col in ["country_name", "year"]:
        continue
    res_d3[f"{col}_filled"] = res_d3.groupby("country_name")[col].transform(fill_missing)

In [ ]:
res_d3.isna().sum()

In [ ]:
res_d3.to_csv("processed_data/preprocessed_d3.csv", index=False)

## Joins

### D1 and D3

In [ ]:
res_d1_d3 = pd.merge(res_d1, res_d3, on=['country_name', 'year'], how='inner')

In [ ]:
res_d1_d3

In [ ]:
res_d1_d3.isna().sum()

In [ ]:
for index, row in res_d1.iterrows():
    c = row['country_name']
    y = row['year']
    if res_d1_d3[(res_d1_d3['country_name'] == c) & (res_d1_d3['year'] == y)].empty:
        print(c, y)

In [ ]:
res_d1_d3.to_csv("processed_data/preprocessed_d1_d3.csv", index=False)

### D2 and D3

In [ ]:
res_d2

In [ ]:
res_d3

In [ ]:
res_d2_d3 = pd.merge(res_d2, res_d3, on=['country_name', 'year'], how='inner')

In [ ]:
res_d2_d3

In [ ]:
res_d2_d3.isna().sum()

In [ ]:
for index, row in res_d2.iterrows():
    c = row['country_name']
    y = row['year']
    if res_d2_d3[(res_d2_d3['country_name'] == c) & (res_d2_d3['year'] == y)].empty:
        print(c, y)

In [ ]:
res_d2_d3.to_csv("processed_data/preprocessed_d2_d3.csv", index=False)

## CEI - Private investment and gross value added (circular economy)

**Dataset**: Circular Economy Indicators (CEI)

**Description**: This dataset tracks the economic performance of circular economy sectors (recycling, repair, reuse) across European countries. It measures:
- **Private investments**: Gross investments in tangible goods.
- **Gross value added**: Economic output of circular economy sectors.

**Unit**: Percentage of Gross Domestic Product (GDP)

**Coverage**: All EU Member States, time series from 2005 onwards

**Preprocessing steps**:
1. Filter for investments as % of GDP
2. Drop countries with >90% missing values
3. Pivot wide format (indicators as columns)
4. Save cleaned dataset

In [ ]:
cei = pd.read_csv("raw_data/cei_cie012.csv")
cei

In [ ]:
res_cei = cei.copy()
res_cei = res_cei.rename(columns={
    "geo": "country_name",
    "TIME_PERIOD": "year",
    "OBS_VALUE": "value",
    "indic_env": "indicator"
})
res_cei = res_cei[res_cei["unit"] == "Percentage of gross domestic product (GDP)"]
res_cei = res_cei[res_cei["country_name"].isin(EU_COUNTRIES)]

res_cei = res_cei[["country_name", "year", "indicator", "value"]]

res_cei["country_name"] = res_cei["country_name"].astype("string")
res_cei["year"] = res_cei["year"].astype(int)
res_cei["indicator"] = res_cei["indicator"].astype("string")
res_cei["value"] = res_cei["value"].astype(float)

print("Unique indicators:", res_cei["indicator"].unique())
print("\nData shape:", res_cei.shape)
res_cei


In [ ]:
# Compute missing rates and drop countries
values_to_drop_cei = []
missing_rates_cei = pd.DataFrame()

for indicator in res_cei["indicator"].unique():
    subset = res_cei[res_cei["indicator"] == indicator]
    mr = pd.DataFrame(subset.groupby("country_name")["value"].apply(lambda x: x.isna().mean())).reset_index()
    mr.columns = ["country_name", "missing_rate"]
    mr["indicator"] = indicator
    
    for _, row in mr.iterrows():
        if row["missing_rate"] > MISSING_RATE:
            values_to_drop_cei.append(row["country_name"])
            print(f"{row['country_name']}\t{indicator}")
    
    missing_rates_cei = pd.concat([missing_rates_cei, mr], ignore_index=True)

missing_rates_cei.to_csv("processed_data/missing_value_rates_cei.csv", index=False)
values_to_drop_cei = list(set(values_to_drop_cei))
print(f"\nDropped countries in CEI (n={len(values_to_drop_cei)}):", values_to_drop_cei)

res_cei = res_cei[~res_cei["country_name"].isin(values_to_drop_cei)]

print(f"\nRemaining countries: {res_cei['country_name'].nunique()}")
print(f"Remaining rows: {len(res_cei)}")

In [ ]:
# Fill missing per (country, indicator) time series
res_cei['value_filled'] = res_cei.groupby(['country_name', 'indicator'])['value'].transform(fill_missing)

# Pivot to wide format: each indicator becomes a column
res_cei_wide = res_cei.pivot_table(
    index=["country_name", "year"],
    columns="indicator",
    values=['value', 'value_filled'],
    aggfunc="first"
).reset_index()

# Flatten multi-level columns
res_cei_wide.columns = [f"{val}_{ind}".replace("_nan", "") if ind else val 
                        for val, ind in res_cei_wide.columns]

print("Final shape:", res_cei_wide.shape)
print("\nColumns:", list(res_cei_wide.columns))
print("\nMissing values:")
print(res_cei_wide.isna().sum())

res_cei_wide.to_csv("processed_data/preprocessed_cei.csv", index=False)
print("\nSaved to processed_data/preprocessed_cei.csv")

## ENV - Environmental tax revenues

**Dataset**: Environmental Tax Revenues (env_ac_tax)

**Description**: This dataset contains government tax revenues from environmental taxes across European countries. Environmental taxes target specific activities with proven negative environmental impact. The dataset contains the total of environmental taxes and taxes on **Pollution & Resources**. This taxes on emissions (air, water), waste management, water abstraction, and raw material extraction

**Unit**: Percentage of Gross Domestic Product (GDP)

**Coverage**: All EU Member States, Iceland, Norway, Switzerland; time series from 1995 onwards

**Preprocessing steps**:
1. Select relevant columns (country, year, tax revenue as % GDP)
2. Drop countries with >90% missing values
3. Apply interpolation and forward/backward fill to handle missing values
4. Save cleaned dataset

In [ ]:
env = pd.read_csv("raw_data/env_ac_tax.csv")
env

In [ ]:
# ENV processing (long-format approach, then pivot)
res_env = env.copy()

# keep relevant columns
res_env = res_env[['geo', 'tax', 'TIME_PERIOD', 'OBS_VALUE']]
res_env = res_env.rename(columns={
    'geo': 'country_name',
    'TIME_PERIOD': 'year',
    'OBS_VALUE': 'value'
})

# types
res_env['country_name'] = res_env['country_name'].astype('string')
res_env['year'] = res_env['year'].astype(int)
res_env['value'] = res_env['value'].astype(float)
res_env['tax'] = res_env['tax'].astype('string')

In [ ]:
# Compute missing rates and drop countries
values_to_drop_env = []
missing_rates_env = pd.DataFrame()

for tax in res_env['tax'].unique():
    subset = res_env[res_env['tax'] == tax]
    mr = pd.DataFrame(subset.groupby('country_name')['value'].apply(lambda x: x.isna().mean())).reset_index()
    mr.columns = ['country_name', 'missing_rate']
    mr['tax'] = tax
    
    for _, row in mr.iterrows():
        if row['missing_rate'] > MISSING_RATE:
            values_to_drop_env.append(row['country_name'])
            print(f"{row['country_name']}\t{tax}")

    missing_rates_env = pd.concat([missing_rates_env, mr], ignore_index=True)

missing_rates_env.to_csv('processed_data/missing_value_rates_env.csv', index=False)
values_to_drop_env = list(set(values_to_drop_env))
print(f"Dropped countries in env (n={len(values_to_drop_env)}):", values_to_drop_env)

# remove those countries entirely
res_env = res_env[~res_env['country_name'].isin(values_to_drop_env)]
print(f"\nRemaining countries: {res_env['country_name'].nunique()}")
print(f"Remaining rows: {len(res_env)}")

In [ ]:
# Fill missing per (country, tax) time series
res_env['value_filled'] = res_env.groupby(['country_name', 'tax'])['value'].transform(fill_missing)

# Pivot to wide format: keep both raw and filled values per tax
res_env_wide = res_env.pivot_table(
    index=['country_name', 'year'],
    columns='tax',
    values=['value', 'value_filled'],
    aggfunc='first'
).reset_index()

# Flatten multi-level columns to readable names and normalize
res_env_wide.columns = [f"{val}_{tax}" if tax else val 
                        for val, tax in res_env_wide.columns]

print('Final shape:', res_env_wide.shape)
print('\nColumns:', list(res_env_wide.columns))
print('\nMissing values:')
print(res_env_wide.isna().sum())

res_env_wide.to_csv('processed_data/preprocessed_env.csv', index=False)
print('\nSaved processed_data/preprocessed_env.csv and processed_data/missing_value_rates_env.csv')

# Result of preprocessing

1. `res_d1`
    1. Dataframe about the basic recycling rates of European countries
    2. Filled values can be found in `recycling_rate_filled` (no missing values)
    3. Dropped no country
    4. Saved missing rates into csv (also the variable: `missing_rates_d1`), be aware, some countries had a lot of missing value
        1. We can later change it to drop a country if it has missing values (or over a %).
    5. Time: 2000-2023
    6. Processed dataframe saved to csv.
2. `res_d2`
    1. Dataframe about the different recycling rates in European countries
    2. Filled values can be found in new column (`recycling_rate_filled_{glass, metallic, packaging, paper, plastic, wooden}`)
    3. Had to drop 7 countries: 100% missing value in one or more categories (most of them had 100% missing in all the categories)
    4. Saved missing rates into csv (also the variable: `missing_rates_d2`), be aware (although better than D1)
        1. We can later change it to drop a country if it has missing values (or over a %).
    5. Time: 2012-2023
    6. Processed dataframe saved to csv.
3. `res_d3`
    1. Dataframe about different indicators for European countries
    2. Filled values can be found in a similar way.
    3. Decided to drop government debt column, because more than 20 countries had it fully missing.
    4. Dropped Bulgaria (100% missing in manufacturing), dropped Kosovo (100% missing in renewable, and tourism)
    5. Saved missing rates into csv (also the variable: `missing_rates_d3`), be aware (although better than D1)
        1. We can later change it to drop a country if it has missing values (or over a %).
    6. Time: 2000-2024
    7. Processed dataframe saved to csv.
4. Joins
    1. D1 and D3 -> `res_d1_d3`
    2. D2 and D3 -> `res_d2_d3`


# Visualizations

First, we visualize the data of `res_d1` (recycling rate) before and after imputation to uncover any trends in the data.

In [ ]:
def plot_country_dashboard(df, primary_metrics, secondary_metric=None, 
                           cols=4, title="Country Analysis"):
    """
    Creates a grid of subplots for countries with support for 
    multiple primary metrics and an optional secondary Y-axis.
    """
    countries = df['country_name'].unique()
    n_countries = len(countries)
    rows = (n_countries // cols) + (1 if n_countries % cols > 0 else 0)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), 
                             sharex=True, sharey=True)
    axes = axes.flatten()

    for i, country in enumerate(countries):
        ax1 = axes[i]
        data = df[df['country_name'] == country].sort_values('year')
        
        # --- Plot Primary Metrics (Shared Left Axis) ---
        for metric, style in primary_metrics.items():
            sns.lineplot(data=data, x='year', y=metric, ax=ax1, 
                         color=style.get('color'), 
                         linestyle=style.get('ls', '-'), 
                         linewidth=style.get('lw', 2), 
                         label=style.get('label', metric), legend=False)
        
        ax1.set_title(country, fontweight='bold', fontsize=14)
        ax1.set_ylim(-2, 100) # Assuming percentage/scaled data
        
        # --- Optional Secondary Metric (Independent Right Axis) ---
        if secondary_metric:
            ax2 = ax1.twinx()
            sns.lineplot(data=data, x='year', y=secondary_metric, ax=ax2, 
                         color='purple', alpha=0.5, lw=1.5, legend=False)
            
            # Formatting right axis labels
            if i % cols != cols - 1:
                ax2.set_yticklabels([])
            else:
                ax2.set_ylabel(secondary_metric.replace('_', ' ').title())

    # Clean up empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Universal Legend
    handles1, labels1 = ax1.get_legend_handles_labels()
    fig.legend(handles1, labels1, loc='upper center', bbox_to_anchor=(0.5, 1.02), 
               ncol=min(len(labels1), 5), fontsize=12, frameon=False)

    plt.tight_layout()
    plt.suptitle(title, fontsize=20, y=1.05, fontweight='bold')
    return fig

In [ ]:
original_v_imputed = {
    'recycling_rate_filled': {'color': 'orange', 'lw': 2, 'alpha': 0.6, 'label': 'Imputed'},
    'recycling_rate': {'color': 'teal', 'lw': 1.5, 'marker': 'o', 'ms': 4, 'label': 'Original'}
}

plot_country_dashboard(res_d1_d3, original_v_imputed, cols=5);

In [ ]:
# Rescaling government effectiveness to 0-100 for better visualization since it is between -2.5 and 2.5
plot_df = res_d1_d3.copy()
plot_df['gov_effectiveness_index'] = ((plot_df['government_effectiveness_estimate_filled'] + 2.5) / 5 * 100)

# Primary metrics
metrics_config = {
    'recycling_rate_filled': {'color': 'orange', 'lw': 3, 'label': 'Recycling %'},
    'urban_population_pct_filled': {'color': 'blue', 'ls': '--', 'label': 'Urban Pop %'},
    'gov_effectiveness_index': {'color': 'black', 'ls': ':', 'label': 'Gov Index'},
    'renewable_energy_pct_filled': {'color': 'green', 'ls': '-.', 'label': 'Renewable Energy %'},
    'gdp_per_capita_filled': {'color': 'purple', 'ls': '-', 'label': 'GDP per Capita'}
}

plot_country_dashboard(plot_df, metrics_config, secondary_metric='gdp_per_capita_filled');

In [ ]:
# Selecting 5 target countries for the management summary
targets = ['Germany', 'Norway', 'Italy', 'Czechia', 'Spain']
df_subset = res_d1_d3[res_d1_d3['country_name'].isin(targets)].copy()

sns.set_theme(style="whitegrid", rc={"axes.facecolor": "#f9f9f9"})
plt.figure(figsize=(12, 7))

palette = sns.color_palette("husl", len(targets))

line_plot = sns.lineplot(
    data=df_subset, 
    x='year', 
    y='recycling_rate_filled', 
    hue='country_name', 
    linewidth=3.5,
    marker='o',
    markersize=8,
    markeredgecolor='white',
    palette=palette
)

# EU 2030 target line
plt.axhline(y=60, color='#d62728', linestyle='--', linewidth=2, alpha=0.8)

plt.text(2000.5, 61, 'EU 2030 Target (60%)', color='#d62728', 
         fontweight='bold', fontsize=12, va='bottom')

plt.title("Path to 2030: Recycling Performance Tracking", 
          fontsize=20, fontweight='bold', pad=25, loc='left')


plt.ylabel("Recycling Rate (%)", fontsize=12, fontweight='bold')
plt.xlabel("Reporting Year", fontsize=12, fontweight='bold')

plt.legend(title="Target Countries", title_fontsize='13', 
           fontsize='11', bbox_to_anchor=(1.02, 1), loc='upper left', frameon=False)

sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis to identify drivers of recycling rates
core_indicators = [
    'recycling_rate_filled',
    'gdp_per_capita_filled',
    'urban_population_pct_filled',
    'renewable_energy_pct_filled',
    'government_effectiveness_estimate_filled']

sns.set_theme(style="white")
plt.figure(figsize=(10, 8))

df_corr = df_subset[core_indicators].corr()
df_corr.columns = [c.replace('_filled', '').replace('_', ' ').title() for c in df_corr.columns]
df_corr.index = [i.replace('_filled', '').replace('_', ' ').title() for i in df_corr.index]


mask = np.triu(np.ones_like(df_corr, dtype=bool))

sns.heatmap(
    df_corr, 
    mask=mask, 
    annot=True, 
    fmt=".2f", 
    cmap='RdYlGn', 
    center=0,
    square=True, 
    linewidths=.5, 
    cbar_kws={"shrink": .7, "label": "Correlation Coefficient ($r$)"},
    annot_kws={"size": 11, "weight": "bold"}
)

plt.title("What Drives Recycling?\nRelationship Analysis for Selected Countries", 
          fontsize=18, pad=25, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Selecting the same 5 target countries for horizontal comparison
targets = ['Germany', 'Norway', 'Italy', 'Czechia', 'Spain']
latest_year = res_d2_d3['year'].max()
material_cols = [c for c in res_d2_d3.columns if 'recycling_rate_filled_' in c]
material_labels = [m.replace('recycling_rate_filled_', '').title() for m in material_cols]

# Plotting horizontal bar charts for each country
fig, axes = plt.subplots(len(targets), 1, figsize=(12, len(targets) * 3), sharex=True)
sns.set_theme(style="whitegrid")

for i, country in enumerate(targets):
    ax = axes[i]
    
    # extract data points
    d2013 = res_d2_d3[(res_d2_d3['country_name'] == country) & (res_d2_d3['year'] == 2013)][material_cols].values.flatten()
    dLatest = res_d2_d3[(res_d2_d3['country_name'] == country) & (res_d2_d3['year'] == latest_year)][material_cols].values.flatten()
    
    y_pos = np.arange(len(material_labels))

    # Show the 2013 baseline
    ax.barh(y_pos, d2013, color='#e0e0e0', label='2013 Rate', height=0.6)
    
    # Show the improvement (green) or decline (red)
    diff = dLatest - d2013
    colors = ['#2ca02c' if x >= 0 else '#d62728' for x in diff]
    ax.barh(y_pos, diff, left=d2013, color=colors, label=f'Change to {latest_year}', height=0.6)

    ax.set_yticks(y_pos)
    ax.set_yticklabels(material_labels, fontweight='bold', fontsize=11)
    ax.set_title(f"Recycling Material Profile: {country}", fontsize=15, fontweight='bold', loc='left', pad=10)
    ax.set_xlim(-2, 100) # Added padding at 0 to see small bars
    
    # data labels
    for j, val in enumerate(dLatest):
        ax.text(val + 1, j, f"{val:.1f}%", va='center', fontsize=10, fontweight='bold')

    sns.despine(left=True, bottom=True)

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.02), ncol=2, frameon=False, fontsize=12)

plt.tight_layout()
plt.show()